# Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Importing the dataset

In [ ]:
product_sale = pd.read_csv('PRODUCT_SALE.csv')
stockname = pd.read_csv('STOCKNAME.csv')
product_sale.sample
product_sale.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,CUSTOMER_ID,BOUGHT_ID,Quantity
0,1,0,1
1,1,4,1
2,1,5,1
3,1,6,1
4,1,7,1
5,1,8,1
6,1,9,1
7,2,0,1
8,2,1,1
9,2,3,1


In [ ]:
stockname.sample
stockname.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,CODE,ITEMNAME
0,1,tape
1,2,pencil
2,3,book
3,4,ruler
4,5,disket
5,6,eraser
6,7,pen
7,8,paper
8,9,liquidpaper
9,0,ink


# Pre-processing

In [ ]:
# Join product_sale and stockname with 'BOUGHT_ID' and 'CODE'
product_sale = pd.merge(product_sale, stockname, left_on='BOUGHT_ID', right_on='CODE', how='left')
product_sale = product_sale.drop(columns=['BOUGHT_ID', 'CODE'])
product_sale.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,CUSTOMER_ID,Quantity,ITEMNAME
0,1,1,ink
1,1,1,ruler
2,1,1,disket
3,1,1,eraser
4,1,1,pen


In [ ]:
product_sale.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2020, 3)

In [ ]:
basket = product_sale.pivot_table(index='CUSTOMER_ID', columns='ITEMNAME', values='Quantity', aggfunc='sum').fillna(0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
basket = basket.astype(bool)
basket

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ITEMNAME,book,disket,eraser,ink,liquidpaper,paper,pen,pencil,ruler,tape
CUSTOMER_ID,,,,,,,,,,
1,False,True,True,True,True,True,True,False,True,False
2,True,True,True,True,True,True,True,False,True,True
3,True,True,True,False,True,False,True,True,False,False
4,True,True,True,True,True,True,True,False,True,True
5,True,True,True,False,True,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...
286,True,True,True,True,True,True,True,True,True,True
287,True,False,True,True,True,True,True,False,True,False
288,True,True,True,True,True,True,False,False,True,True


# Construct model using Association Rules

## Apply FP-growth to determine all the frequent itemsets. Set minimum support value to 80%

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(basket, min_support=0.8, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.948276,(book)
1,0.810345,(ink)
2,0.924138,(liquidpaper)
3,0.944828,(paper)
4,0.882759,"(book, liquidpaper)"
5,0.896552,"(book, paper)"
6,0.872414,"(paper, liquidpaper)"
7,0.834483,"(paper, book, liquidpaper)"


## Apply “Create Association Rules” operators. Set confidence value to 90%

In [ ]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.9)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(book),(liquidpaper),0.948276,0.924138,0.882759,0.930909,1.007327,0.006421,1.098004,0.140625
1,(liquidpaper),(book),0.924138,0.948276,0.882759,0.955224,1.007327,0.006421,1.155172,0.095881
2,(book),(paper),0.948276,0.944828,0.896552,0.945455,1.000664,0.000595,1.011494,0.012821
3,(paper),(book),0.944828,0.948276,0.896552,0.948905,1.000664,0.000595,1.012315,0.012019
4,(paper),(liquidpaper),0.944828,0.924138,0.872414,0.923358,0.999156,-0.000737,0.989819,-0.015085
5,(liquidpaper),(paper),0.924138,0.944828,0.872414,0.944030,0.999156,-0.000737,0.985747,-0.011016
6,"(book, paper)",(liquidpaper),0.896552,0.924138,0.834483,0.930769,1.007176,0.005945,1.095785,0.068871
7,"(liquidpaper, paper)",(book),0.872414,0.948276,0.834483,0.956522,1.008696,0.007194,1.189655,0.067568
8,"(book, liquidpaper)",(paper),0.882759,0.944828,0.834483,0.945312,1.000513,0.000428,1.008867,0.004375
9,(paper),"(book, liquidpaper)",0.944828,0.882759,0.834483,0.883212,1.000513,0.000428,1.003879,0.009298


# Model deployment using Association Rules

## List all the frequent itemsets that have been found (with their support)

In [ ]:
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.948276,(book)
1,0.810345,(ink)
2,0.924138,(liquidpaper)
3,0.944828,(paper)
4,0.882759,"(book, liquidpaper)"
5,0.896552,"(book, paper)"
6,0.872414,"(paper, liquidpaper)"
7,0.834483,"(paper, book, liquidpaper)"


## What is the maximal frequent itemset found? Give its support? List all the strong association rules that can be generated from it.

In [ ]:
rules.sort_values(by='confidence', ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
7,"(liquidpaper, paper)",(book),0.872414,0.948276,0.834483,0.956522,1.008696,0.007194,1.189655,0.067568
1,(liquidpaper),(book),0.924138,0.948276,0.882759,0.955224,1.007327,0.006421,1.155172,0.095881
3,(paper),(book),0.944828,0.948276,0.896552,0.948905,1.000664,0.000595,1.012315,0.012019
2,(book),(paper),0.948276,0.944828,0.896552,0.945455,1.000664,0.000595,1.011494,0.012821
8,"(book, liquidpaper)",(paper),0.882759,0.944828,0.834483,0.945312,1.000513,0.000428,1.008867,0.004375
5,(liquidpaper),(paper),0.924138,0.944828,0.872414,0.944030,0.999156,-0.000737,0.985747,-0.011016
0,(book),(liquidpaper),0.948276,0.924138,0.882759,0.930909,1.007327,0.006421,1.098004,0.140625
6,"(book, paper)",(liquidpaper),0.896552,0.924138,0.834483,0.930769,1.007176,0.005945,1.095785,0.068871
4,(paper),(liquidpaper),0.944828,0.924138,0.872414,0.923358,0.999156,-0.000737,0.989819,-0.015085
11,(liquidpaper),"(book, paper)",0.924138,0.896552,0.834483,0.902985,1.007176,0.005945,1.066313,0.093914


Maximal frequent itemset is (paper, book, liquidpaper) and is (liquidpaper, paper) => (book) with confidence of 0.956522

## According to the rule {liquidpaper} => {book,paper}, explain its meaning

(liquidpaper) => (book,paper) means if customers buy liquidpaper, they have a chance buy book and paper with confidence of 0.902985	and there are 83.4483 % of users to buy all (liquidpaper,book,paper)

## What items are expected to be bought by customers who buy liquidpaper?

book with confidence of 0.955224 and paper with confidence of 0.944030 or buy both with confidence of 0.902985